In [5]:
import pandas as pd
import os
import warnings
import numpy as np
import re
import json

from functools import reduce
from datetime import datetime
from datetime import date

warnings.filterwarnings("ignore")

### ========================================================

In [6]:
test_0909_uni = ['bodyType', 'brand', 'color', 'engineDisplacement', 'enginePower', 
                'equipment_dict','fuelType', 'mileage', 'modelDate', 'model_name', 'numberOfDoors', 
                'productionDate', 'vehicleTransmission', 'vendor', 
                'Владельцы', 'Владение', 'ПТС', 'Привод', 'Руль']

### ========================================================

In [7]:
test = pd.read_csv('./Project_7_data/test.csv',low_memory=False)
df= pd.read_csv('./Project_7_data/' + 'auto_ru_2020_09_09.csv',low_memory=False)

### ========================================================

In [8]:
def bodyType_unification(x):
    res=[]
    x = x.lower() if type(x) == str else x  # <================
    try:
        for body_type in test.bodyType.unique():
            if body_type in x:
                res.append(body_type)
    except Exception: return x
    return max(res)

def engineDisplacement_unification(x):
    x = float(re.sub("[^\d.]", r'', x)) if re.sub("[^\d.]", r'', x) else 0
    if x >= 7: x = 0
    return x

def train_options(x):
    point = "'available_options': "
    start = x.find(point)+len(point)+2
    finish = x.find("]",start) - 1
    return x[start:finish].split("', '")

def test_ownership_unification(x):
    try:
        digits = re.findall('\d+',x) 
        if len(digits) == 2: res = int(digits[0])*12 + int(digits[1])
        elif len(digits) == 1 and 'месяц' in x: res = int(digits[0])
        elif len(digits) == 1 and 'месяц' not in x: res = int(digits[0])*12
    except Exception: res = 0   
    return res

def train_ownership_unification(x):
    tmp = json.loads(x.replace("'",'"'))  if x==x else {'year': 2020, 'month': 9}
    res = (2020 - tmp['year'])*12 + tmp['month'] - 9
    if res<0: res = 0
    return res


In [9]:
def test_unification(df_to_proc):
    test = df_to_proc.copy()
    owner_dict = {'3 или более': 3., '2\xa0владельца': 2.,'1\xa0владелец': 1.}
    # engineDisplacement
    test.engineDisplacement = test.engineDisplacement.apply(
        lambda x: float(x[:-4]) if x[:-4] else np.NaN) 
    # enginePower
    test.enginePower = test.enginePower.apply(lambda x: int(x[:-4]))
    # equipment_dict
    test.equipment_dict = test.equipment_dict.apply(
        lambda x: list(json.loads(x).keys()) if x==x else [])
    # Владельцы
    test.Владельцы = test.Владельцы.map(owner_dict)
    # Владение
    test.Владение = test.Владение.apply(test_ownership_unification)
    return test

In [10]:
def train_0909_unification(df_to_proc):
    df = df_to_proc.copy()
    color_codes = {'040001': 'чёрный','FAFBFB': 'белый', '0000CC': 'синий', 
                   '200204': 'коричневый', 'EE1D19': 'красный', 'CACECB': 'серый',
                   'C49648': 'бежевый', '97948F': 'серебристый', 'FFD600': 'жёлтый',
                   'FF8649': 'оранжевый', '22A0F8': 'голубой','FFC0CB': 'розовый', 
                   'DEA522': 'золотистый', '007F00': 'зелёный', '660099': 'пурпурный',
                   '4A2197': 'фиолетовый'}
    transmission_dict = {'MECHANICAL':'механическая', 'AUTOMATIC':'автоматическая', 
                         'ROBOT':'роботизированная','VARIATOR':'вариатор'}
    vendor_dict = {'AUDI':'EUROPEAN','BMW':'EUROPEAN','HONDA':'JAPANESE','INFINITI':'JAPANESE',
                   'LEXUS':'JAPANESE','MERCEDES':'EUROPEAN','MITSUBISHI':'JAPANESE',
                   'NISSAN':'JAPANESE','SKODA':'EUROPEAN','TOYOTA':'JAPANESE',
                   'VOLKSWAGEN':'EUROPEAN','VOLVO':'EUROPEAN'}
    PTS_dict = {'ORIGINAL': 'Оригинал', 'DUPLICATE': 'Дубликат'}
    wheel_dict = {'LEFT':'Левый', 'RIGHT':'Правый'}
    # bodyType
    df.dropna(subset=['bodyType'],inplace=True)
    df.bodyType = df.bodyType.apply(bodyType_unification)
    # color
    df.color = df.color.map(color_codes)
    # engineDisplacement
    df.engineDisplacement = df.engineDisplacement.apply(engineDisplacement_unification)
    # enginePower
    df.enginePower = df.enginePower.astype(int)
    # equipment_dict
    df['equipment_dict'] = df.Комплектация.apply(train_options)
    df.drop(columns=['Комплектация'],inplace=True)
    # modelDate 
    df.modelDate = df.modelDate.astype(int)
    # model_name
    df['model_name'] = df.model
    df.drop(columns=['model'],inplace=True)
    # numberOfDoors
    df.numberOfDoors = df.numberOfDoors.astype(int)
    # vehicleTransmission
    df.vehicleTransmission = df.vehicleTransmission.map(transmission_dict)
    # vendor
    df['vendor'] = df.brand.map(vendor_dict)
    # Владение
    df.Владение = df.Владение.apply(train_ownership_unification)
    # ПТС
    df.ПТС = df.ПТС.map(PTS_dict)
    # Руль
    df.Руль = df.Руль.map(wheel_dict)
    
    return df

In [11]:
%%time 
test_unification(test)

,bodyType,brand,car_url,color,complectation_dict,description,engineDisplacement,enginePower,equipment_dict,fuelType,...,vehicleConfiguration,vehicleTransmission,vendor,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня
0,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,синий,NaN,"Все автомобили, представленные в продаже, прох...",1.2,105,"[engine-proof, tinted-glass, airbag-driver, au...",бензин,...,LIFTBACK ROBOT 1.2,роботизированная,EUROPEAN,3.0,0,Оригинал,передний,Левый,Не требует ремонта,Растаможен
1,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,чёрный,NaN,ЛОТ: 01217195\nАвтопрага Север\nДанный автомоб...,1.6,110,"[cruise-control, asr, esp, airbag-driver, isof...",бензин,...,LIFTBACK MECHANICAL 1.6,механическая,EUROPEAN,1.0,0,Оригинал,передний,Левый,Не требует ремонта,Растаможен
2,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/superb/11...,серый,"{""id"":""20026336"",""name"":""Ambition"",""available_...","Все автомобили, представленные в продаже, прох...",1.8,152,"[cruise-control, tinted-glass, esp, adaptive-l...",бензин,...,LIFTBACK ROBOT 1.8,роботизированная,EUROPEAN,1.0,0,Оригинал,передний,Левый,Не требует ремонта,Растаможен
3,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,коричневый,"{""id"":""20803582"",""name"":""Ambition"",""available_...",КОМПЛЕКТ ЗИМНЕЙ (ЛЕТНЕЙ) РЕЗИНЫ ПО СЕЗОНУ В ПО...,1.6,110,"[cruise-control, roller-blind-for-rear-window,...",бензин,...,LIFTBACK AUTOMATIC 1.6,автоматическая,EUROPEAN,1.0,0,Оригинал,передний,Левый,Не требует ремонта,Растаможен
4,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,белый,NaN,ЛОТ: 01220889\nАвтопрага Север\n\nВы можете по...,1.8,152,"[cruise-control, asr, esp, airbag-driver, isof...",бензин,...,LIFTBACK AUTOMATIC 1.8,автоматическая,EUROPEAN,1.0,0,Оригинал,передний,Левый,Не требует ремонта,Растаможен
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34681,седан,BMW,https://auto.ru/cars/used/sale/bmw/3er/1101369...,коричневый,NaN,Продается BMW 316i в отличном состоянии. Прода...,1.6,136,"[cruise-control, asr, tinted-glass, esp, start...",бензин,...,SEDAN AUTOMATIC 1.6,автоматическая,EUROPEAN,3.0,25,Оригинал,задний,Левый,Не требует ремонта,Растаможен
34682,седан,BMW,https://auto.ru/cars/used/sale/bmw/5er/1101369...,чёрный,NaN,Продаётся отличный автомобиль.,2.0,190,[],дизель,...,SEDAN AUTOMATIC 2.0,автоматическая,EUROPEAN,1.0,0,Оригинал,полный,Левый,Не требует ремонта,Растаможен
34683,седан,BMW,https://auto.ru/cars/used/sale/bmw/5er/1101364...,серый,NaN,Хорошее состояние,2.5,170,[],бензин,...,SEDAN AUTOMATIC 2.5,автоматическая,EUROPEAN,3.0,0,Дубликат,задний,Левый,Не требует ремонта,Растаможен
34684,внедорожник 5 дв.,BMW,https://auto.ru/cars/used/sale/bmw/x1/11013625...,коричневый,"{""id"":""6336008"",""name"":""xDrive20d Локальная сб...",Автомобиль в идеальном состоянии . Куплен новы...,2.0,184,"[engine-proof, esp, start-stop-function, airba...",дизель,...,ALLROAD_5_DOORS AUTOMATIC 2.0,автоматическая,EUROPEAN,2.0,0,Оригинал,полный,Левый,Не требует ремонта,Растаможен


In [12]:
%%time
train_0909_unification(df)

,bodyType,brand,color,fuelType,modelDate,name,numberOfDoors,productionDate,vehicleConfiguration,vehicleTransmission,...,Владельцы,ПТС,Таможня,Владение,price,start_date,hidden,equipment_dict,model_name,vendor
0,седан,AUDI,чёрный,бензин,1990,2.8 MT (174 л.с.) 4WD,4,1991,SEDAN MECHANICAL 2.8,механическая,...,3.0,Оригинал,True,0,200000.0,2019-10-03T08:09:11Z,NaN,[],100,EUROPEAN
1,седан,AUDI,красный,бензин,1982,1.8 MT (90 л.с.),4,1986,SEDAN MECHANICAL 1.8,механическая,...,3.0,Оригинал,True,0,60000.0,2020-09-06T06:49:40Z,NaN,[],100,EUROPEAN
2,универсал 5 дв.,AUDI,синий,бензин,1988,2.3 MT (136 л.с.) 4WD,5,1989,WAGON_5_DOORS MECHANICAL 2.3,механическая,...,3.0,Оригинал,True,0,99000.0,2020-09-02T14:04:21Z,NaN,[],100,EUROPEAN
3,седан,AUDI,серый,бензин,1988,1.8 MT (90 л.с.),4,1989,SEDAN MECHANICAL 1.8,механическая,...,3.0,Оригинал,True,0,65000.0,2020-08-23T17:40:09Z,NaN,[],100,EUROPEAN
4,седан,AUDI,чёрный,бензин,1990,2.0 MT (101 л.с.),4,1991,SEDAN MECHANICAL 2.0,механическая,...,3.0,Дубликат,True,0,100000.0,2020-09-08T09:22:07Z,NaN,[],100,EUROPEAN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49304,купе,MERCEDES,золотистый,бензин,1951,3.0 MT (150 л.с.),2,1952,COUPE MECHANICAL 3.0,механическая,...,1.0,Оригинал,True,195,10800000.0,2019-01-24T08:50:28Z,NaN,[],W188,EUROPEAN
49305,седан,MERCEDES,чёрный,бензин,1957,3.0 AT (160 л.с.),4,1959,SEDAN AUTOMATIC 3.0,автоматическая,...,2.0,Оригинал,True,0,5600000.0,2020-06-20T16:13:54Z,NaN,[],W189,EUROPEAN
49306,пикап двойная кабина,MERCEDES,серебристый,дизель,2017,350 d 3.0d AT (258 л.с.) 4WD,4,2018,PICKUP_TWO AUTOMATIC 350,автоматическая,...,1.0,Оригинал,True,15,4000000.0,2020-03-03T14:56:44Z,NaN,[],X_KLASSE,EUROPEAN
49307,пикап двойная кабина,MERCEDES,серебристый,дизель,2017,250 d 2.3d AT (190 л.с.) 4WD,4,2018,PICKUP_TWO AUTOMATIC 250,автоматическая,...,1.0,Оригинал,True,0,3150000.0,2020-08-05T07:36:53Z,NaN,"[cruise-control, heated-wash-system, airbag-pa...",X_KLASSE,EUROPEAN
